<a href="https://colab.research.google.com/github/climate-and-health-datasci-Unicamp/ldl-cholesterol-climate-analysis/blob/master/notebooks/02_LDL_lag_transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Authors:**
* [Júlia Perassolli De Lázari](https://github.com/juliaplazari)
* [Thiago Ribas Bella](https://github.com/trbella)

*created in jun/2020*

## LDL lag transformation

INPUTS:

- [df_clean.csv](https://media.githubusercontent.com/media/climate-and-health-datasci-Unicamp/data/master/LMC/interim/df_clean.csv),

- [cold_wave_IAC.csv](https://media.githubusercontent.com/media/climate-and-health-datasci-Unicamp/data/master/LMC/interim/cold_wave_IAC.csv),

- [heat_wave_IAC.csv](https://media.githubusercontent.com/media/climate-and-health-datasci-Unicamp/data/master/LMC/interim/heat_wave_IAC.csv)

OUTPUT: 
* [ldl_waves_lags.csv](https://media.githubusercontent.com/media/climate-and-health-datasci-Unicamp/data/master/LMC/processed/ldl_waves_lags.csv)

The dataframe ldl_waves_lags has the following columns: results of ldl exam, date of exam, sex, age, age group and presence or absence of cold and heat waves from 0 to 10 days after ldl exam realization.

The dataframe goes from 2008 to 2018 (withouth leap days).



### Read and prepare data

Read the data from df_clean

The dataframe is prepared to be analysed with climate dataframes:

- Remove rows with leap days (2012-02-29 and 2016-02-29)
- Remove rows with sex as 'I'
- Create age group category
- Remove rows with 2019 data 

The data of 2019 is not used as it's not complete.



In [15]:
#@title Load data and libraries
#-------------------------------------------------------------------#
#                       Import libraries                            #
#-------------------------------------------------------------------#
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#-------------------------------------------------------------------#
#                      Load clean dataframe                         #
#-------------------------------------------------------------------#
url = 'https://media.githubusercontent.com/media/climate-and-health-datasci-Unicamp/data/master/LMC/interim/df_clean.csv'
df_clean = pd.read_csv(url)
print("Lenght of clean dataframe: ",len(df_clean))
df_clean.head()

df_clean = df_clean.drop(columns = ['Unnamed: 0','Year','Month','Day'])
df_clean = df_clean.rename(columns = {'DateTimeExam':'DATE'})
df_clean.sort_values(by = 'DATE').head()



Lenght of clean dataframe:  1038281


,Procedure,DATE,Parameter,Result,Sex,City,UF,Age
412,LDL,2008-03-03,LDLP,98.0,F,CAMPINAS,SAO PAULO,39
333,LDL,2008-03-03,LDLP,143.0,M,CAMPINAS,SAO PAULO,59
491,LDL,2008-03-03,LDLP,179.0,M,CAMPINAS,SAO PAULO,56
492,LDL,2008-03-03,LDLP,166.0,F,CAMPINAS,SAO PAULO,41
493,LDL,2008-03-03,LDLP,178.0,F,CAMPINAS,SAO PAULO,67


In [16]:
#@title Create age group category
df_clean.loc[df_clean['Age'] < 2.0, 'Age group'] = '1 - Infant'
df_clean.loc[(df_clean['Age'] >=2.0) & (df_clean['Age'] <= 12.0), 'Age group'] = '2 - Child'
df_clean.loc[(df_clean['Age'] >=13.0) & (df_clean['Age'] <= 19.0), 'Age group'] ='3 - Teenager'
df_clean.loc[(df_clean['Age'] >=20.0) & (df_clean['Age'] <= 39.0), 'Age group'] ='4 - Young adult'
df_clean.loc[(df_clean['Age'] >=40.0) & (df_clean['Age'] <= 64.0), 'Age group'] ='5 - Adult'
df_clean.loc[df_clean['Age'] >64.0, 'Age group'] = '6 - Elderly'

In [17]:
#@title Remove rows with sex as 'I'
#-------------------------------------------------------------------#
#                         Most relevant data                        #
#-------------------------------------------------------------------#

df = df_clean[['Result', 'DATE','Age','Age group','Sex']] #most relevant data
print("There're", len(df[(df.DATE =='2012-02-29')&(df.DATE =='2016-02-29')]), "rows with leap year dates")
df = df[(df.DATE !='2012-02-29')&(df.DATE !='2016-02-29')] #remove leap year dates (02-29)
print("There're", len(df[df['Sex'] == 'I']), "rows with sex = 'I'")
df = df[df['Sex']!= 'I'] #remove rows with sex = 'I'
print("Lenght of the new dataframe: ",len(df))

There're 0 rows with leap year dates
There're 10392 rows with sex = 'I'
Lenght of the new dataframe:  1026882


In [18]:
#@title Remove rows with 2019 data
#reset index
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.set_index('DATE')
print(df.tail(5))

#remove 2019 data
print('\n')
print("There're", len(df.loc['2019-01-01':'2019-05-31']),"rows with 2019 data")
df = df.loc['2008-03-03':'2019-01-01']
df = df.reset_index()
print("Lenght of the dataframe:",len(df))


            Result  Age        Age group Sex
DATE                                        
2019-05-31   137.0   69      6 - Elderly   F
2019-05-31   154.0   45        5 - Adult   M
2019-05-31   103.0   38  4 - Young adult   F
2019-05-31   106.0   75      6 - Elderly   M
2019-05-31   137.0   70      6 - Elderly   M


There're 46836 rows with 2019 data
Lenght of the dataframe: 980046


# **Cold waves and heat waves data**

Get cold and heat waves data and merge it with the LDL dataframe.

## **Dataset: cold_wave_IAC**
**below_pct**

Whether the day had temperatures above 10th percentile. [See supplement](https://github.com/climate-and-health-datasci-Unicamp/ldl-cholesterol-climate-analysis/blob/master/references/How_do_we_calculate_thermal_waves.docx?raw=true).

**Column cwave**

inicio = when the cold wave started

meio = during cold wave

fim = next day after cold wave ended

0 = control day

**Column ci** (binary)

1 = start of cold wave

**Column cf** (binary)

1 = end of cold wave

### Below we will load cold wave data from Meteorological Station of Agronomic Institute of Campinas (IAC) and merge with cholesterol dataset.

In [19]:
#@title Load cold wave data and merge with cholesterol dataset
#-------------------------------------------------------------------#
#                          Cold wave data                           #
#-------------------------------------------------------------------#
urlcold = 'https://media.githubusercontent.com/media/climate-and-health-datasci-Unicamp/data/master/LMC/interim/cold_wave_IAC.csv'
cw = pd.read_csv(urlcold, sep = ';')
cw.DATE = pd.to_datetime(cw.DATE,dayfirst=True)
print(cw.head(5))
#print(cw.DATE.head().dt.day)

#beginning of cold wave
c_1 = cw[cw['cwave']=='inicio']
print(np.unique(cw['cwave'].astype(str)))
print(c_1.head(5))

#merge with LDL dataframe
df2 = pd.merge(df,cw,on='DATE')
print(df2.sort_values(by='DATE').head())


        DATE  below_pct  CW cwave  ci  cf
0 1961-01-01          0   0     0   0   0
1 1961-01-02          0   0     0   0   0
2 1961-01-03          0   0     0   0   0
3 1961-01-04          0   0     0   0   0
4 1961-01-05          0   0     0   0   0
['0' 'fim' 'inicio' 'meio']
           DATE  below_pct  CW   cwave  ci  cf
1145 1964-02-20          1   1  inicio   1   0
1301 1964-07-26          1   1  inicio   1   0
1889 1966-03-06          1   1  inicio   1   0
3763 1971-04-24          1   1  inicio   1   0
3973 1971-11-20          1   1  inicio   1   0
          DATE  Result  Age        Age group Sex  below_pct  CW cwave  ci  cf
395 2008-03-03   104.0   64        5 - Adult   F          0   0     0   0   0
479 2008-03-03    60.0   34  4 - Young adult   M          0   0     0   0   0
480 2008-03-03   205.0   56        5 - Adult   F          0   0     0   0   0
481 2008-03-03   125.0   45        5 - Adult   F          0   0     0   0   0
482 2008-03-03   128.0   60        5 - Adult   F

## **Dataset: heat_wave_IAC**
**above_pct** (binary)

Whether the day had temperatures below 90th percentile. [See supplement](https://github.com/climate-and-health-datasci-Unicamp/ldl-cholesterol-climate-analysis/blob/master/references/How_do_we_calculate_thermal_waves.docx?raw=true).

**Column hwave**

inicio = when the heat wave started

meio = during heat wave

fim = next day after heat wave ended

0 = control day

**Column ci** (binary)

1 = start of heat wave

**Column cf** (binary)

1 = end of heat wave

### Below we will load heat wave data from Meteorological Station of Agronomic Institute of Campinas (IAC) and merge with merged dataset above.

In [20]:
#@title Load heat wave data
#-------------------------------------------------------------------#
#                          Heat wave data                           #
#-------------------------------------------------------------------#
urlheat = 'https://media.githubusercontent.com/media/climate-and-health-datasci-Unicamp/data/master/LMC/interim/heat_wave_IAC.csv'
hw = pd.read_csv(urlheat,sep=';')
hw.DATE = pd.to_datetime(hw.DATE,dayfirst=True)
print(hw.head(5))
h_1 = hw[hw['hwave']=='inicio'] #datas onde começa onda de frio

df3 = pd.merge(df2,hw)
df3.sort_values(by='DATE').head()

        DATE  above_pct  HW hwave  hi  hf
0 1961-01-01          0   0     0   0   0
1 1961-01-02          0   0     0   0   0
2 1961-01-03          0   0     0   0   0
3 1961-01-04          0   0     0   0   0
4 1961-01-05          0   0     0   0   0


,DATE,Result,Age,Age group,Sex,below_pct,CW,cwave,ci,cf,above_pct,HW,hwave,hi,hf
395,2008-03-03,104.0,64,5 - Adult,F,0,0,0,0,0,0,0,0,0,0
479,2008-03-03,60.0,34,4 - Young adult,M,0,0,0,0,0,0,0,0,0,0
480,2008-03-03,205.0,56,5 - Adult,F,0,0,0,0,0,0,0,0,0,0
481,2008-03-03,125.0,45,5 - Adult,F,0,0,0,0,0,0,0,0,0,0
482,2008-03-03,128.0,60,5 - Adult,F,0,0,0,0,0,0,0,0,0,0


In [21]:
#@title Count of days with thermal waves
#-------------------------------------------------------------------#
#                    Number of cold and heat waves                  #
#-------------------------------------------------------------------#

c1 = len(np.unique(df3['DATE'][df3['cwave']=='inicio']))
h1 = len(np.unique(df3['DATE'][df3['hwave']=='inicio']))
n1 = len(np.unique(df3['DATE'][(df3['hwave']==0)&(df3['cwave']==0)]))

print('Thermal waves present in LDL exam')
print(f'Number of cold waves in LDL results = {c1}')
print(f'Number of heat waves in LDL results = {h1}')
print(f'Days without thermal waves (control group) in LDL results = {n1}')

Thermal waves present in LDL exam
Number of cold waves in LDL results = 7
Number of heat waves in LDL results = 18
Days without thermal waves (control group) in LDL results = 0


### Dataframe with thermal waves and lagged data

New dataframe with thermal waves and lagged ldl results data, which goes from  0 to 10 days after the thermal wave beginning.

In [22]:
#@title Thermal wave and ldl lagged data
#-------------------------------------------------------------------#
#                 Dataframe with termal waves and lag               #
#-------------------------------------------------------------------#
#   function lag for dates
#   pandas.Index.shift
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.shift.html

df_shift = df3
df_shift = df_shift.set_index('DATE') #date column as index
dates = pd.DataFrame(df.DATE) #dataframe with LDL exam results dates
dates = dates.set_index('DATE') #date column as index

for lag in range(0,-11,-1):
    name = lag*(-1)
    dates = dates.shift(lag,freq='D') #dates with lag for the date dataframe

# Verify if the date of ldl exam (dates.index) was the beginning of 
# a thermal wave (c_1['DATE'])
# 1 if yes, 0 otherwise

    df_shift['bool'] = np.isin(dates.index,c_1['DATE'])
    df_shift[f'CI_lag{name}'] = np.where(df_shift['bool']==True,1,0)
    df_shift['bool'] = np.isin(dates.index,h_1['DATE'])
    df_shift[f'HI_lag{name}'] = np.where(df_shift['bool']==True,1,0)
    dates = pd.DataFrame(df3.DATE)
    dates = dates.set_index('DATE')

df_shift = df_shift.reset_index()
df_shift.sort_values(by='DATE').head()

,DATE,Result,Age,Age group,Sex,below_pct,CW,cwave,ci,cf,above_pct,HW,hwave,hi,hf,bool,CI_lag0,HI_lag0,CI_lag1,HI_lag1,CI_lag2,HI_lag2,CI_lag3,HI_lag3,CI_lag4,HI_lag4,CI_lag5,HI_lag5,CI_lag6,HI_lag6,CI_lag7,HI_lag7,CI_lag8,HI_lag8,CI_lag9,HI_lag9,CI_lag10,HI_lag10
395,2008-03-03,104.0,64,5 - Adult,F,0,0,0,0,0,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
479,2008-03-03,60.0,34,4 - Young adult,M,0,0,0,0,0,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
480,2008-03-03,205.0,56,5 - Adult,F,0,0,0,0,0,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
481,2008-03-03,125.0,45,5 - Adult,F,0,0,0,0,0,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
482,2008-03-03,128.0,60,5 - Adult,F,0,0,0,0,0,0,0,0,0,0,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Save the dataframe


In [23]:
#@title Save the dataframe as ldl_waves_lags.csv
df_shift.to_csv('ldl_waves_lags.csv')
!cp ldl_waves_lags.csv '/content/drive/Shared drives/Clima&Saúde/Dados/Dados_Saude/Clinicos_LMC/LDL/SUBSET'

cp: cannot create regular file '/content/drive/Shared drives/Clima&Saúde/Dados/Dados_Saude/Clinicos_LMC/LDL/SUBSET': No such file or directory
